In [1]:
from phylodeep import BD, BDEI, BDSS, FULL
from phylodeep.modeldeep import modeldeep
from phylodeep.paramdeep import paramdeep


path_to_tree = "/Users/mmcanear/Projects/PhD_Courses/STAT700/phylodeep/test_tree_HIV_Zurich/Zurich.trees"

# set presumed sampling probability
sampling_proba = 0.25

# NOTE: Absolutely does not work due to broadcasting issues
# a priori check for models BD, BDEI, BDSS
# checkdeep(path_to_tree, model=BD, outputfile_png='BD_a_priori_check.png')
# checkdeep(path_to_tree, model=BDEI, outputfile_png='BDEI_a_priori_check.png')
# checkdeep(path_to_tree, model=BDSS, outputfile_png='BDSS_a_priori_check.png')


# model selection
model_BDEI_vs_BD_vs_BDSS = modeldeep(
    path_to_tree, sampling_proba, vector_representation=FULL
)

# the selected model is BDSS

# parameter inference
param_BDSS = paramdeep(
    path_to_tree,
    sampling_proba,
    model=BDSS,
    vector_representation=FULL,
    ci_computation=True,
)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step


# Encoding

Exploratory analysis of how to actually get the CBLV representation in Phylodeep; this is NOT strictly provided by the package, it seems.

In [ ]:
from phylodeep.encoding import encode_into_most_recent 
from ete3 import Tree
from pathlib import Path
import pandas as pd
from tqdm import tqdm
from matplotlib import pyplot as plt
import re

output_dir = Path("../output_trees")

param_files = list(output_dir.glob("*.csv"))
all_tree_files = list(output_dir.glob("*.nwk"))

tree_indices = [int(re.sub("[^0-9]", "", f.name)) for f in all_tree_files]
param_indices = [int(re.sub("[^0-9]", "", f.name)) for f in param_files]


In [127]:
sampling_proba = 0.5

def read_tree(tree_file, idx):
    try:
        with open(tree_file, "r") as f:
            nwk_str = f.readline().strip()   
        tree = Tree(nwk_str, format=1)
        tree_encoding, _ = encode_into_most_recent(tree, 0.5)
        
        tree_encoding["idx"] = idx
        tree_encoding = tree_encoding.set_index("idx")
        return tree_encoding

    except Exception as e:
        # print(f"Error reading {tree_file}: {e}")
        return None

all_tree_files = list(output_dir.glob("*.nwk"))
all_tree_outputs = [read_tree(f, idx) for f, idx in tqdm(zip(all_tree_files, tree_indices), total=len(tree_indices))]


100%|██████████| 6858/6858 [01:34<00:00, 72.81it/s] 


In [130]:
tree_df = pd.concat(all_tree_outputs)

In [132]:
tree_df.to_csv("./tree_encodings.csv")

In [ ]:
import re

def _skip_reader(file_path, idx):
    try:
        df = pd.read_csv(file_path)
        df["idx"] = idx
        df.set_index("idx", inplace=True)
        return df
    except Exception as e:
        return pd.DataFrame()


valid_params = [
    _skip_reader(f, idx) for f, idx in tqdm(zip(param_files, param_indices), total=len(param_files))
]
assert len(valid_params) == len(param_indices)
param_df = pd.concat(valid_params)
# plt.scatter(params["la_II"], params["psi_I"], s=0.5)

100%|██████████| 8656/8656 [00:07<00:00, 1100.63it/s]


In [ ]:
# tree_df.fillna(0, inplace=True)


400     False
1        True
3       False
5       False
7       False
        ...  
992     False
994     False
996     False
1001    False
998     False
Length: 1004, dtype: bool